In [1]:
from gurobipy import *

model = Model ('AirplaneCargo')

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-14


In [2]:
# ---- Parameters ----


# Cargo characteristics
cargoname     = ('bulk_1', 'bulk_2', 'bulk_3', 'bulk_4')
cargovolume   = (500, 700, 600, 400)        # m3 / ton
cargoprofit   = (320, 400, 360, 290)        # euro / ton

NrOfDays = 3                                # planning horizon 
                                            # (model uses only the cargodata of the days in this range)
cargoquantity = ( ( 20, 0, 0),              # ton available each day
                  ( 16, 0, 0),              # not transported cargo is available for next days
                  ( 25, 0, 0), 
                  ( 13, 0, 0) )

                  
# Compartment characteristics
compname  = ('front', 'center', 'back')
maxweight = (12, 18, 10)                    # ton
maxvolume = (7000, 9000, 5000)              # m3



In [3]:
# ---- Sets ----

I = range (len (cargoname) )                # set of cargo types
J = range (len (compname) )                 # set of compartments
K = range (NrOfDays)                        # set of compartments



In [4]:
# ---- Variables ----

# Decision Variable x(i,j) (cargo of type in in compartment j)
x = {} 
for i in I:
    for j in J:
        for k in K:
            x[i,j,k] = model.addVar (lb = 0, vtype = GRB.CONTINUOUS, name = 'X[' + str(i) + ',' + str(j) + ',' + str(k) + ']')
# Integrate new variables
model.update ()

In [5]:
# ---- Objective Function ----

model.setObjective (quicksum (cargoprofit[i] * x[i,j,k] for i in I for j in J for k in K) )
model.modelSense = GRB.MAXIMIZE
model.update ()

In [6]:
# Constraints 1: volume capacity
con1 = {}
for j in J:
    for k in K:
        con1[j,k] = model.addConstr( quicksum (cargovolume[i] * x[i,j,k] for i in I) <= maxvolume[j], 'con1[' + str(j) + ',' + str(k) + ']-')

# Constraints 2: weight capacity
con2 = {}
for j in J:
    for k in K:
        con2[j,k] = model.addConstr( quicksum (x[i,j,k] for i in I) <= maxweight[j], 'con2[' + str(j) + ',' + str(k) + ']-')

# Constraints 3: available amount 
con3 = {}
for i in I:
    for k in K:
        con3[i,k] = model.addConstr( quicksum (x[i,j,d] for j in J for d in range (k+1) ) <= quicksum (cargoquantity[i][d] for d in range (k+1)), 'con3[' + str(i) + ',' + str(k) + ']-')


# Constraint 4: weight balance first compartment with other compartments
con4 = {}
for k in K:
    for j in range (1, len(J)):
        con4[k,j] = model.addConstr( quicksum (x[i,0,k] for i in I) * maxweight[j] == quicksum (x[i,j,k] for i in I) * maxweight[0], 'con4[' + str(j) + ']-')


In [7]:
# ---- Solve ----

model.setParam( 'OutputFlag', True) # silencing gurobi output or not
model.setParam ('MIPGap', 0);       # find the optimal solution
model.write("output.lp")            # print the model in .lp format file

model.optimize ()

Set parameter MIPGap to value 0
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 36 rows, 36 columns and 192 nonzeros
Model fingerprint: 0x717d4368
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [3e+02, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 9e+03]
Presolve time: 0.01s
Presolved: 36 rows, 36 columns, 192 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8531250e+32   1.310859e+32   3.853125e+02      0s
      19    2.5570000e+04   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.557000000e+04


In [8]:
# --- Print results ---
print ('\n--------------------------------------------------------------------\n')
    
if model.status == GRB.Status.OPTIMAL: # If optimal solution is found
    print ('Total profit : %10.2f euro' % model.objVal)
    print ('')
    print ('All decision variables:\n')
    
    for k in K:
        s = 'Day %d   ' % (k+1)
        for i in I:
            s = s + '%8s' % cargoname[i]
        print (s)    

        for j in J:
            s = '%8s' % compname[j]
            for i in I:
                s = s + '%8.3f' % x[i,j,k].x
            s = s + '%8.3f' % sum (x[i,j,k].x for i in I)    
            print (s)    

        s = '%8s' % ''
        for i in I:
            s = s + '%8.3f' % sum (x[i,j,k].x for j in J)    
        print (s)
        print ('\n')    

else:
    print ('\nNo feasible solution found')

print ('\nREADY\n')



--------------------------------------------------------------------

Total profit :   25570.00 euro

All decision variables:

Day 1     bulk_1  bulk_2  bulk_3  bulk_4
   front   0.717   0.000   0.000   0.000   0.717
  center   0.000   0.000   1.076   0.000   1.076
    back   0.000   0.598   0.000   0.000   0.598
           0.717   0.598   1.076   0.000


Day 2     bulk_1  bulk_2  bulk_3  bulk_4
   front   0.000   9.483   0.000   0.000   9.483
  center   0.000   0.000  14.224   0.000  14.224
    back   0.000   2.586   5.316   0.000   7.902
           0.000  12.069  19.540   0.000


Day 3     bulk_1  bulk_2  bulk_3  bulk_4
   front  12.000   0.000   0.000   0.000  12.000
  center   7.283   0.000   4.384   6.333  18.000
    back   0.000   3.333   0.000   6.667  10.000
          19.283   3.333   4.384  13.000



READY

